In [1]:
import os
import sys
import numpy as np
import scipy 
import scipy.stats
import random
import string
import itertools

In [2]:
trace_props_small = {
               "max_keysize": 8, 
               "max_valsize": 128, 
               "num_keys": 128, 
                "num_requests": 512, 
               "valsize_dist": "zipf",
                "outfile": "trace_small.c"       
              }

In [3]:
trace_props_long = {
               "max_keysize": 8, 
               "max_valsize": 128, 
               "num_keys": 128, 
                "num_requests": 1024, 
               "valsize_dist": "constant",
                "outfile": "trace_long.c"       
              }

In [4]:
trace_props_big = {
               "max_keysize": 16, 
               "max_valsize": 128, 
               "num_keys": 1024, 
                "num_requests": 2048, 
               "valsize_dist": "constant",
                "outfile": "trace_big.c"       
              }

In [5]:
trace_props_large = {
               "max_keysize": 16, 
               "max_valsize": 1024, 
               "num_keys": 1024, 
                "num_requests": 2048, 
               "valsize_dist": "constant",
                "outfile": "trace_large.c"       
              }

In [6]:
trace_props_xlarge = {
               "max_keysize": 64, 
               "max_valsize": 1024, 
               "num_keys": 2048, 
                "num_requests": 4196, 
               "valsize_dist": "constant",
                "outfile": "trace_xlarge.c"       
              }

In [7]:
def generate_test_file(trace_props):
    global db
    db = []
    output_header = "#include<kv.h> \n int main(){ \n \
    int retval; \n     int set_errors = 0; \n \
    char* valtmp=NULL; \n     int get_errors=0; \n    kv_init(); \n"
    output_trailer = '\n    printf("%d, %d \\n",set_errors, get_errors); \n kv_reset(); \n } \n'
    load_data(trace_props)
    set_output = cnvrt_set(db)
    
    get_output = cnvrt_get(issue_gets())
    
    out = output_header + set_output + get_output + output_trailer
    
    f = open(trace_props.get("outfile"), "w+")
    f.write(out)
    f.close()
    

In [8]:
def gen_kv_pair(keysize=8, max_valsize=16, valsize_dist="constant"):
    zrv = scipy.stats.zipf(1.5, loc=max_valsize/4)
    if valsize_dist is "constant":
        valsize = max_valsize-1 
    elif valsize_dist is "zipf":
        valsize = min(max_valsize, zrv.rvs())
    
    key = ''.join(random.choice(string.ascii_lowercase) for _ in range(keysize))
    val = ''.join(random.choice(string.ascii_lowercase) for _ in range(valsize))
    #valstr * (max_valsize-1)
    
    return (key, val)

In [9]:
def load_data(trace_props):
    global db 
    db = []
    num_keys = trace_props.get("num_keys")
    for _ in range(num_keys):
        (key,val) = gen_kv_pair(trace_props.get("max_keysize"),\
                                trace_props.get("max_valsize"), \
                                trace_props.get("valsize_dist"))
        db.append((key,val))
        

In [10]:
def cnvrt_set(db):
    outstr = "\n"
    for (key,val) in db:
        outstr = outstr + "\n" + "    retval=kv_set(\"{}\", \"{}\"); \n \
    if(retval>0) set_errors++; \n ".format(key, val)
    return outstr+"\n"

In [11]:
#print(cnvrt_set(db))

In [12]:
def issue_gets(num_requests=100, dist="zipf"):
    get_list = [] 
    num_keys = len(db)
    
    if dist is "uniform":
        key_indices = np.random.randint(num_keys)
    else:
        key_indices = np.random.zipf(2, num_keys)

    key_iterator = itertools.cycle(key_indices)
    for _ in range(num_requests):
        i = next(key_iterator)
        get_list.append(db[i])
    
    return get_list 

In [13]:
def cnvrt_get(get_list):
    outstr = ""
    for (key, val) in get_list:
        outstr = outstr + "\n" + "    valtmp = kv_get(\"{}\"); \n \
    if(valtmp==NULL) get_errors++; \n".format(key)
    return outstr

In [14]:
valstr  = 'v'
db = []
generate_test_file(trace_props_small)
generate_test_file(trace_props_long)
generate_test_file(trace_props_big)
generate_test_file(trace_props_large)
generate_test_file(trace_props_xlarge)